# Reduce dataset size
---

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
from subprocess import Popen               # Run shell commands
import pandas as pd                        # Pandas to load the data initially

## Initializing variables

In [ ]:
bucket_name = input('Bucket name:')

## Detecting which unit stays end up in death

Go one by one in each unit stay's file and build a dictionary which indicates which contain positive samples (i.e. death happening at least 96h after the last sample in the unit stay data).

Create a dictionary that indicates, for each unit stay file, if it's positive (i.e. death happening at least 96h after the last sample in the unit stay data) or negative:

In [ ]:
ends_dead = dict()

In [ ]:
for i in range(8000):
    # Download and read the file
    Popen(f'gsutil cp gs://bucket_name/eICU_{i}', shell=True).wait()
    df = pd.read_feather(f'eICU_{i}.ftr')
    # Check if it's a positive or negative case (dead or alive)
    # 5760 minutes = 96 hours
    if df.death_ts.max() <= df.ts.max() + 5760:
        ends_dead[f'eICU_{i}.ftr'] = True
    else:
        ends_dead[f'eICU_{i}.ftr'] = False
    # Delete the file from disk
    Popen(f'rm -rf eICU_{i}', shell=True).wait()

Save the dictionary:

## Separating into train, validation and test sets

Since I want to reduce the dataset size to 1/10 (i.e. to a total of 800 unit stays) and have each set contain approximately the same death ratio, I'll use the dictionary created above to get 576 training unit stays (0.9x0.8x800), 64 validation unit stays (0.1x0.8x800) and 160 test unit stays (0.2x800).

Save the train, validation and test indeces (i.e. the lists of file names):